In [ ]:
import numpy as np   
from scipy.special import erf
import random
from matplotlib import pyplot as plt
from matplotlib.pyplot import cm 
%matplotlib inline
import GPflow.kernels
from GPflow.likelihoods import Bernoulli, Gaussian
from GPflow.svgp import SVGP
from GPflow.svgp_additive import SVGP_additive2 


In this notebook we are interested in fitting generalized additive model.
The generative model is the following

$$\begin{align}
f_d &\sim GP(0,k_d(\theta_d)),\quad \forall d \in [1,D]\\
\eta^{(i)} &= \sum_d f_d(x^{(i)}_d)\\
y^{(i)} &\sim p(y|\eta^{(i)}) \quad \text{some known likelihood
}.
\end{align}$$

Here, $x_d \in \cal{X}_d$ and $\cap_d \cal{X}_d= \phi$ , which just means that subsets of covariates for different functions are non overlapping

We follow Hensman 2015 (Scalable Variational Gaussian Process Classification) and extend the Posterior approximation: 

* each Gaussian Process is augmented with inducing points $u_d$ associated to pseudo-inputs $Z_d$

* A factorized approximation to the posterior over functions is chosen

$$\begin{align}
p(f_1,...,f_D,u_1,...,u_D|Y,X,Z) &\approx \prod_d q(f_d,u_d)\\
&\approx \prod_d p(f_d|u_d)q(u_d).
\end{align}$$

* Factors over inducing points are chosen to be Gaussian: $q(u_d)=\cal{N}(\mu_d,\Sigma_d)$

* We construct a lower bound on the model evidence

$$\log \,p(Y) \geq \sum_i \mathbb{E}_{\rho_i} \log p(y_i|\rho_i) +\sum_d KL(q(u_d)||p(u_d))$$

where $\rho_i$ is a univariated Gaussian whose sufficient statistics are functions of $Z,\mu,\Sigma, \theta$

* Approximate inference is performed by maximizing the lower bound with respect to
 * the inducing point locations $Z_d$
 * the associated variational parameters $\mu_d,\Sigma_d$
 * the kernel hyperparameters $\theta_d$
 

## Creating a dataset

In [ ]:
np.random.seed(1)

N = 1000 # number of data points
D = 3 # number of covariate dimension
X = np.random.rand(N, D)-.5 # sampling covariates uniformly

# some arbitrary functions    
f1 = lambda x : x
f2 = lambda x : np.sin(x*5.)
f3 = lambda x : -10.*x*np.exp(-np.abs(x)*5.)

fns = [f1] # forcing the first covariate to be linear 
for d in range(1,D):
    fns += [random.choice([f2,f3])]
    
# Computing the additive predictor
Fs = np.zeros((N,D))
for d in range(D):
    Fs[:,d] = fns[d](X[:,d])
F = np.sum(Fs,axis=1).reshape(N,1)

# Computing the observables

lik = 'Gaussian'
#lik = 'Bernoulli'

if lik == 'Gaussian':
    s_n = 1
    Y = F + np.random.randn(N,1) * s_n
elif lik == 'Bernoulli':
    phi =lambda x: 0.5*(1+erf(x/np.sqrt(2)))
    B = phi(F)
    Y = (np.random.rand(N,1)<B).astype(int)



### Plotting dataset

In [ ]:
# plotting individual functions
xp = np.linspace(-.5,.5,200)
for d in range(D):
    plt.plot(xp,fns[d](xp),'-')
plt.title('Individual functions')
plt.xlabel('$x_d$',fontsize=20)
plt.ylabel('$f_d(x_d)$',fontsize=20)
plt.show()


# histogram of predictor values
plt.title('Predictor values')
plt.hist(F)
plt.xlabel('$\sum_d f_d$',fontsize=20)
plt.show()

# Classification only: histogram of Bernoulli parameters
if lik == 'Bernoulli':
    plt.title('Bernoulli parameters values')
    plt.hist(phi(F))
    plt.xlabel('$\phi ( \sum_d f_d )$',fontsize=20)
    plt.show()


### Setting up model

In [ ]:
# Inducing point locations 
Nz = 20
Z = [np.array([[1]])] # one pseudo input for linear term
for d in range(D-1):
    Z+= [np.random.rand(Nz, 1)-.5] # list of (M,1) array
    
# Setting likelihood
if lik == 'Gaussian':
    likelihood = Gaussian()
    likelihood.variance = 0.01
elif lik == 'Bernoulli':
    likelihood = Bernoulli()

# Setting kernels
ks = [GPflow.kernels.Linear(1)]
for k in range(1,D):
    ks += [ GPflow.kernels.RBF(1)]


# Declaring model
m = SVGP_additive2(X, Y, ks, likelihood, Z)


### Fixing parameters
Here, we decide which parameters we want to optimize. These include
* kernel hyperparameters $\theta$
* inducing point locations $Z$
* variational parameters
* likelihood parameters (if any)

In [ ]:
# --- Kernel parameters
for k in m.kerns.parameterized_list:
    #if k.name == 'linear':
    #    k.variance.fixed = True
    #if k.name == 'rbf':
    #    k.variance.fixed = True
    #    k.lengthscales.fixed = True
    pass
        
# --- Inducing points
m.Z[0].fixed = True # no need to optimize location for linear parameter
#for z in m.Z:
#    z.fixed=True

# --- Likelihood parameters
if lik == 'Gaussian':
    #m.likelihood.variance.fixed = True
    pass


# --- Variational parameters
#for qmu in m.q_mu:
    #qmu.fixed = True
#for qs in m.q_sqrt:
    #qs.fixed = True
    

### Running optimization

In [ ]:
# optimizing
for k in range(2):
    m.optimize()

### Diagnosis

In [ ]:
# computing predicted sum (mean and variance)
Ds = range(D)
m.set_prediction_subset_ds(Ds)

Fp, VFp = m.predict_f(X)
Yp, VYp = m.predict_y(X)
SFp = np.sqrt(VFp)
SYp = np.sqrt(VYp)

# computing RootMeanSquaredError 
rmse = np.sqrt(np.mean((Yp - Y) ** 2))
print "RMSE: %.3f" % rmse

# plotting true against inferred predictor
n = 100 #subselect plots
I = np.random.randint(1,len(Y),n)
fig,ax = plt.subplots()
ax.errorbar(F,Fp , yerr=np.sqrt(SFp), fmt='o')
lims = [ np.min([ax.get_xlim(), ax.get_ylim()]),
         np.max([ax.get_xlim(), ax.get_ylim()])]
ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
ax.set_xlabel('underlying predictor',fontsize=20)
ax.set_ylabel('estimated predictor mean',fontsize=20)
plt.show()
# Bernoulli case

fig,ax = plt.subplots()
if lik == 'Bernoulli':
    # bin responses per bernoulli param
    ax.errorbar(B,Yp , yerr=np.sqrt(SYp), fmt='o')
    ax.set_xlabel('true $\phi(\sum f)$',fontsize=20)
    ax.set_ylabel('predicted $\phi(\sum f)$',fontsize=20)
plt.show()

In [ ]:
print X.shape
print m.prediction_ds
import tensorflow as tf
print tf.expand_dims(X[:, m.f_indices[2]],1)

### Display results

In [ ]:
# Generating predictions for individual functions
Ys=[]
Vs=[]
for d in range(D):
    m.set_prediction_subset_ds([d])
    Yd, Vd = m.predict_f(X)
    Ys.append(Yd)
    Vs.append(Vd)

In [ ]:
col=cm.rainbow(np.linspace(0,1,D))
fig1,ax1 = plt.subplots()
w = 5
fig2,axarr = plt.subplots(1,D,figsize=(D*w,D))


# plotting infered functions against true functions
for d in range(D):
    Yd = Ys[d]
    Vd = Vs[d]
    o = np.argsort(X[:,d])
    ax1.plot(X[o,d],Fs[o,d],'--',linewidth=4,c=col[d])
    ax1.plot(X[o,d],Yd[o],'-',c=col[d])
    ax1.fill_between(X[o,d],
                     y1=np.squeeze(Yd[o]+np.sqrt(Vd[o])),
                     y2=np.squeeze(Yd[o]-np.sqrt(Vd[o])),facecolor=col[d],alpha=.5)

    
# plotting infered functions against true functions
for d in range(D):
    Yd = Ys[d]
    Vd = Vs[d]
    ax=axarr[d]
    ax.errorbar(Fs[o,d], Yd[o], yerr=np.sqrt(Vd), fmt='o')
    lims = [ np.min([ax.get_xlim(), ax.get_ylim()]),
             np.max([ax.get_xlim(), ax.get_ylim()])]
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_xlabel('underlying predictor',fontsize=20)
    ax.set_ylabel('estimated predictor mean',fontsize=20)
plt.show()



In [ ]:
x = np.zeros((10,10))
print x[:,np.array([1,2])].shape